# Exercise 2.1

In [2]:

!grep ">" data/salmonella_spi1_region.fna

>gi|821161554|gb|CP011428.1| Salmonella enterica subsp. enterica strain YU39, complete genome, subsequence 3000000 to 3200000


In [3]:
def read_fasta(filename):
    """
    Read a FASTA File
    Assumes first line is the descriptor and the rest is
    the sequence
    """
    with open(filename, 'r') as f:
        descriptor = f.readline().rstrip()
        
        sequence = ""
        line = f.readline().rstrip()
        print(sequence)
        while line != "":
            sequence += line
            line = f.readline().rstrip()
            
    return descriptor, sequence

In [4]:
descriptor, seq = read_fasta('data/salmonella_spi1_region.fna')

In [5]:
print(descriptor)

>gi|821161554|gb|CP011428.1| Salmonella enterica subsp. enterica strain YU39, complete genome, subsequence 3000000 to 3200000


In [6]:
seq[:500]

'AAAACCTTAGTAACTGGACTGCTGGGATTTTTCAGCCTGGATACGCTGGTAGATCTCTTCACGATGGACAGAAACTTCTTTCGGGGCGTTCACGCCAATACGCACCTGGTTGCCCTTCACCCCTAAAACTGTCACGGTGACCTCATCGCCAATCATGAGGGTCTCACCAACTCGACGAGTCAGAATCAGCATTCTTTGCTCCTTGAAAGATTAAAAGAGTCGGGTCTCTCTGTATCCCGGCATTATCCATCATATAACGCCAAAAAGTAAGCGATGACAAACACCTTAGGTGTAAGCAGTCATGGCATTACATTCTGTTAAACCTAAGTTTAGCCGATATACAAAACTTCAACCTGACTTTATCGTTGTCGATAGCGTTGACGTAAACGCCGCAGCACGGGCTGCGGCGCCAACGAACGCTTATAATTATTGCAATTTTGCGCTGACCCAGCCTTGTACACTGGCTAACGCTGCAGGCAGAGCTGCCGCATCCGTACCAC'

# Exercise 2.2

In [8]:
descriptor, seq = read_fasta('data/lambda.fasta.txt')

In [9]:
print(descriptor)

>Lambda_NEB


In [36]:
seq[:500]

'GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACCCTCTGAAAAGAAAGGAAACGACAGGTGCTGAAAGCGAGGCTTTTTGGCCTCTGTCGTTTCCTTTCTCTGTTTTTGTCCGTGGAATGAACAATGGAAGTCAACAAAAAGCAGCTGGCTGACATTTTCGGTGCGAGTATCCGTACCATTCAGAACTGGCAGGAACAGGGAATGCCCGTTCTGCGAGGCGGTGGCAAGGGTAATGAGGTGCTTTATGACTCTGCCGCCGTCATAAAATGGTATGCCGAAAGGGATGCTGAAATTGAGAACGAAAAGCTGCGCCGGGAGGTTGAAGAACTGCGGCAGGCCAGCGAGGCAGATCTCCAGCCAGGAACTATTGAGTACGAACGCCATCGACTTACGCGTGCGCAGGCCGACGCACAGGAACTGAAGAATGCCAGAG'

In [34]:
def restriction_sites(seq, recog_seq):
    # Create list to add index of cut sites
    cut_sites = []
    
    # I'm dumb, I could've done this in a much easier way
    for i in range (len(seq) - len(recog_seq)):
        correct_bases = 0
        for j, recog_base in enumerate(recog_seq):
            
            if seq[i+j] == recog_base:
                correct_bases += 1
            if correct_bases == len(recog_seq):
                cut_sites.append(i)
                
                
    return cut_sites

In [42]:
print('HindIII:', restriction_sites(seq, 'AAGCTT'))
print('EcoRI:  ', restriction_sites(seq, 'GAATTC'))
print('KpnI:   ', restriction_sites(seq, 'GGTACC'))

HindIII: [23129, 25156, 27478, 36894, 37458, 37583, 44140]
EcoRI:   [21225, 26103, 31746, 39167, 44971]
KpnI:    [17052, 18555]


In [43]:
def restriction_sites_better(seq, recog_seq):
    cut_sites = []
    
    for i in range (len(seq) - len(recog_seq)):
        if seq[i:i+len(recog_seq)] == recog_seq:
            cut_sites.append(i)
    
    return cut_sites

In [44]:
print('HindIII:', restriction_sites_better(seq, 'AAGCTT'))
print('EcoRI:  ', restriction_sites_better(seq, 'GAATTC'))
print('KpnI:   ', restriction_sites_better(seq, 'GGTACC'))

HindIII: [23129, 25156, 27478, 36894, 37458, 37583, 44140]
EcoRI:   [21225, 26103, 31746, 39167, 44971]
KpnI:    [17052, 18555]


# Exercise 2.3

## (a)

In [52]:
def gc_content(seq):
    # If the sequence is empty, return 0.0
    if seq == '':
        return 0.0;
    
    # Make string all uppercase
    seq.upper()
    
    # Create variable for the number of gc bases
    gc_count = 0
    
    # Loop to count number of Cs and Gs
    # I forgot that strings have a count method
    for base in seq:
        if (base == 'G') or (base == 'C'):
            gc_count += 1
            
        
    return gc_count / len(seq)


In [53]:
gc_content('')

0.0

In [56]:
gc_content('GCGCGCAAAG')

0.7

In [57]:
def gc_blocks(seq, block_size):
    # Create empty list to store the blocks' GC content
    block_gc_content = []
    
    for i in range(0, len(seq) - (len(seq) % block_size), block_size):
        block_gc_content.append(gc_content(seq[i:i+block_size]))
    
    return tuple(block_gc_content)

In [58]:
gc_blocks('ATGACTACGT', 4)

(0.25, 0.5)

## (b)

In [86]:
def gc_map(seq, block_size, gc_thresh):
    """ 
    Takes a sequence a block size and a GC
    threshold
    Returns the original sequence with blocks
    with GC content above threshold in caps
    and blocks below GC content in lowercase
    
    
    I definitely could've done this in a better
    way but this works
    """
    
    # Make sure whole sequence is upper case
    seq.upper()
    
    # Create empty output string
    mapped_seq = ''
    
    # Get gc content of blocks
    gc_content_blocks = gc_blocks(seq, block_size)
    
    for i, gc in enumerate(gc_content_blocks):
        if gc >= gc_thresh:
            mapped_seq += seq[i*block_size:(i+1)*block_size]
        else:
            mapped_seq += seq[i*block_size:(i+1)*block_size].lower()
            
    return mapped_seq

In [87]:
gc_map('ATGACTACGT', 4, 0.4)

'atgaCTAC'

In [90]:
descriptor, seq = read_fasta('data/salmonella_spi1_region.fna')

In [91]:
print(descriptor)

>gi|821161554|gb|CP011428.1| Salmonella enterica subsp. enterica strain YU39, complete genome, subsequence 3000000 to 3200000


In [92]:
seq[:500]

'AAAACCTTAGTAACTGGACTGCTGGGATTTTTCAGCCTGGATACGCTGGTAGATCTCTTCACGATGGACAGAAACTTCTTTCGGGGCGTTCACGCCAATACGCACCTGGTTGCCCTTCACCCCTAAAACTGTCACGGTGACCTCATCGCCAATCATGAGGGTCTCACCAACTCGACGAGTCAGAATCAGCATTCTTTGCTCCTTGAAAGATTAAAAGAGTCGGGTCTCTCTGTATCCCGGCATTATCCATCATATAACGCCAAAAAGTAAGCGATGACAAACACCTTAGGTGTAAGCAGTCATGGCATTACATTCTGTTAAACCTAAGTTTAGCCGATATACAAAACTTCAACCTGACTTTATCGTTGTCGATAGCGTTGACGTAAACGCCGCAGCACGGGCTGCGGCGCCAACGAACGCTTATAATTATTGCAATTTTGCGCTGACCCAGCCTTGTACACTGGCTAACGCTGCAGGCAGAGCTGCCGCATCCGTACCAC'

In [93]:
salmonella_mapped = gc_map(seq, 10000, 0.45)

In [95]:
#salmonella_mapped

In [100]:
with open('data/salmonella_spi1_region.fna', 'w') as f:
    # Write the description
    f.write(descriptor + '\n')
    
    i = 0
    while i < len(salmonella_mapped) - 59:
        f.write(salmonella_mapped[i:i+60] + '\n')
        i += 60
    
    f.write(salmonella_mapped[i:] + '\n')
    

In [101]:
!head data/salmonella_spi1_region.fna

>gi|821161554|gb|CP011428.1| Salmonella enterica subsp. enterica strain YU39, complete genome, subsequence 3000000 to 3200000
AAAACCTTAGTAACTGGACTGCTGGGATTTTTCAGCCTGGATACGCTGGTAGATCTCTTC
ACGATGGACAGAAACTTCTTTCGGGGCGTTCACGCCAATACGCACCTGGTTGCCCTTCAC
CCCTAAAACTGTCACGGTGACCTCATCGCCAATCATGAGGGTCTCACCAACTCGACGAGT
CAGAATCAGCATTCTTTGCTCCTTGAAAGATTAAAAGAGTCGGGTCTCTCTGTATCCCGG
CATTATCCATCATATAACGCCAAAAAGTAAGCGATGACAAACACCTTAGGTGTAAGCAGT
CATGGCATTACATTCTGTTAAACCTAAGTTTAGCCGATATACAAAACTTCAACCTGACTT
TATCGTTGTCGATAGCGTTGACGTAAACGCCGCAGCACGGGCTGCGGCGCCAACGAACGC
TTATAATTATTGCAATTTTGCGCTGACCCAGCCTTGTACACTGGCTAACGCTGCAGGCAG
AGCTGCCGCATCCGTACCACCGGCTTGCGCCATGTCCGGACGACCGCCACCCTTACCGCC


In [ ]:
!tail data/salmonella_spi1_region.fna

# Exercise 2.4